In [1]:
from datasets import load_dataset, Dataset, DatasetDict
from collections import Counter
import numpy as np
import pandas as pd
from datasets import Dataset, Features, Value

In [ ]:
"""
This notebook will allow you to generate synthetic dataset based on phi and psi parameter (which defines how much is the verfier is utilised)
"""

In [2]:
out_dir = "output"
data = pd.read_json(f"{out_dir}/full_responses_guard.json")

In [ ]:
len(data)

In [4]:
def extract_resp(msg):
    assistant_content = msg[1]["content"]
    resp = assistant_content[assistant_content.find("Agent:\n")+7:]
    return resp

In [5]:
def add_resp(row):
    row["response"] = extract_resp(row["messages"])
    return row
data = data.apply(add_resp, axis=1)

In [ ]:
data["llm_judge"].value_counts()

In [ ]:
data["Guard"].value_counts()

In [8]:
df_agree = data[data["llm_judge"]==data["Guard"]]

In [9]:
df_disagree = data[~(data["llm_judge"]==data["Guard"])]

In [ ]:
print(f"agree = {len(df_agree)}\ndisagree = {len(df_disagree)}")

In [11]:
phi = 1  ## prob selected if llm_judge = Guard (verfier) 
psi = 1    ## prob selected if llm_judge != Guard (verfier)
org = "<replace with you HF org>"
HF_path = f"{org}/safety-synthetic-v2-weak"

In [12]:
def filter_phi(row):
    return np.random.choice([False, True],p=[1-phi,phi])
def filter_psi(row):
    return np.random.choice([False, True],p=[1-psi,psi])

In [13]:
df_agree_f = df_agree[df_agree.apply(filter_phi,axis=1)]

In [14]:
df_disagree_f = df_disagree[df_disagree.apply(filter_psi,axis=1)]

In [ ]:
print(f"agree_f = {len(df_agree_f)}\ndisagree_f = {len(df_disagree_f)}")

In [16]:
df_f = pd.concat([df_agree_f,df_disagree_f]).sample(frac=1)[["prompt","response","llm_judge","Guard"]]

In [ ]:
df_f.head()

In [18]:
sys_prompt = "You will be given an interaction between 'Human' and 'Agent'. Your task is to classfiy if the 'agent' response is safe or unsafe."

In [19]:
def prepare_sft(row):
     row["input"] = f"Human:\n {row['prompt']}\n Agent:\n {row['response']}"
     row["messages"] = [{"content": sys_prompt,"role":"system"}, {"content": row['input'],"role":"user"}, {"content":row['llm_judge'].upper(),"role":"assistant"}]
     row["category"] = row["llm_judge"]
     return row

In [20]:
df_train_sft = df_f.apply(prepare_sft, axis=1)[["input","messages","category"]]

In [ ]:
df_train_sft

In [ ]:
hf_train_sft = Dataset.from_pandas(df_train_sft)

hf_test_sft = load_dataset("PKU-Alignment/PKU-SFT-Safety", split="test")

In [ ]:
hf_train_sft

In [ ]:
hf_test_sft = hf_test_sft.select(range(10))
hf_test_sft

In [25]:
SFT_dataset = DatasetDict({
    'train': hf_train_sft,
    'test': hf_test_sft
})

In [ ]:
SFT_dataset

In [ ]:
SFT_dataset.push_to_hub(HF_path,private=True)

In [ ]:
load_dataset(f"{org}/safety-synthetic-ph10-ps10")